In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
from scipy.integrate import solve_ivp


## Orbit solution

In [ ]:
def big_system(t, Y_M, f, Gradf, dim, A, a, b, gamma, eta):
    # Solving numerically the initial value problem (dy/dt,dM/dt = (f(t,y),Gradf*M) 
    X_M = np.zeros((dim+dim**2)) #We solve simustanuously d+d*d ODEs to link up the solution and the monodromy matrix computation
    M = Y_M[dim:].reshape(dim, dim, order="F")  # Reshape the flat array back into a dim^2 x dim matrix
    dM_dt = Gradf(t,Y_M[:dim], A, a, b, gamma, eta) @ M  # Compute the matrix derivative
    X_M[:dim] = f(t, Y_M[:dim],A,a,b,gamma,eta)
    X_M[dim:] = dM_dt.flatten(order = "F")
    return X_M

In [ ]:
def Newton_orbit(f,s, X_0,T_0,A, a, b, gamma, eta, Gradf,GradS,ds_T, Max_iter, epsilon):
    #Ici on resoud tout le system en assemblant la grande matrice par bloque
    dim = np.shape(X_0)[0] #The problem dimension

    #________________________________INITIALISATION___________________________________________________________
    k, Xstar_0, Tstar = 0, X_0, T_0
    # sol = solve_ivp(fun=f,t_span=[0.0, T_0], y0=X_0, t_eval=np.linspace(0.0, Tstar, 10),
    #                  method='RK45', args = params, option={"jac": Gradf, "rtol": 1e-6,"atol":1e-9})#,"max_step": max_step})
    # G_Xstar_0 = sol.y[:,-1] - Xstar_0 #We take the solution at t=T

    # #The complete rhd of the Newton system
    # B = np.concatenate((G_Xstar_0, np.array([s(Tstar,Xstar_0, params[0], params[3])])))

    X_by_iter, T_by_iter = np.zeros((Max_iter,dim)),np.zeros((Max_iter))
    Norm_B = Norm_DeltaX = np.zeros((Max_iter))
    Norm_DeltaX[0] = norm_delta_X = 1 # 
    # Norm_B[0] = np.linalg.norm(B)

    #_____________Newton iteration loop________________
    while norm_delta_X > epsilon  and k < Max_iter:
        print("Iteration", k, "\n")
        print("Norm(Dx) = ", norm_delta_X,"\n")
        X_by_iter[k,:] = Xstar_0
        Norm_DeltaX[k] = norm_delta_X
                
        X_by_iter[k,:] = Xstar_0
        T_by_iter[k] = Tstar


        X_M = np.zeros((dim+dim**2)) #We solve simustanuously d+d*d ODEs
        monodromy = np.eye(dim) #Initialisation of the monodromy matrix

        X_M[:dim] = Xstar_0
        X_M[dim:] = monodromy.flatten(order='F')
        big_sol= solve_ivp(big_system, [0.0,Tstar], X_M,
                            t_eval=[Tstar],
                            args=(f,Gradf,dim,A, a, b, gamma, eta),
                            method='RK45',**{"rtol": 1e-6,"atol":1e-9}) #It's a function of t
        
        Xstar_T = big_sol.y[:dim,-1] #dimension d+d*d
        monodromy = big_sol.y[dim:][:,-1] #We take M(T)

        monodromy = monodromy.reshape(dim,dim, order = "F") #Back to the square matrix format
        d = ds_T(Xstar_0, Tstar)
        c = GradS(Tstar,Xstar_0, gamma)
        bb = f(Tstar, Xstar_T, A, a, b, gamma, eta)
        #Concat the whole matrix
        top = np.hstack((monodromy - np.eye(dim), bb.reshape(-1,1)))  # Horizontal stacking of A11=M-I and A12=b
        bottom = np.hstack((c.reshape(1,-1),np.array([[d]])))  # Horizontal stacking of A21=c and A22=d
        Mat = np.vstack((top, bottom))  # Vertical stacking of the two rows
        
        #Right hand side concatenation
        B = np.concatenate((Xstar_T - Xstar_0, np.array([s(Tstar,Xstar_0, A,gamma)])))
        XX = solve(Mat,-B) #Contain Delta_X and Delta_T
        #XX = np.linalg.lstsq(Mat, -B, rcond=None)[0]
        Delta_X = XX[:dim]
        Delta_T = XX[-1]
        
        #Updating
        Xstar_0 += Delta_X
        Tstar += Delta_T

        norm_delta_X = np.linalg.norm(Delta_X)
        Norm_B[k] = np.linalg.norm(B)

        k = k+1
        
    # Compute the eigenvalues (Floquet multipliers) of the monodromy matrix
    # eigenvalues, eigenvectors = np.linalg.eig(monodromy)
    return k, T_by_iter, X_by_iter, Norm_B, Norm_DeltaX, monodromy#, eigenvalues


### APPLICATION TO THE LASER EQUATION

In [ ]:
K = lambda a,b,X: 1 + a/(1+b*X)
def f(t,X, A, a, b, gamma, eta): 
        return np.array([(X[1] - K(a,b,X[0]))*X[0] + eta*X[1],
                                  (A-X[1]-X[1]*X[0])*gamma])

Gradf = lambda t,X,A,a,b,gamma,eta: np.array([[a*b*X[0]/((1+b*X[0])**2) + X[1] - K(a,b,X[0]), 
                                                 X[0]+ eta],
 
                                                 [-gamma*X[1],-gamma*(1+X[0])]])



# Variable chang L = log(I)
fL = lambda t,X,A,a,b,gamma,eta: np.array([(X[1] - K(a,b,np.exp(X[0]))) + eta*X[1]/np.exp(X[0]),
                                  (A-X[1]-X[1]*np.exp(X[0]))*gamma])

GradfL = lambda t,X,A,a,b,gamma,eta: np.array([[a*b*np.exp(X[0])/(1+b*np.exp(X[0]))**2 -eta*X[1]*np.exp(-X[0]), 
                                                 1 + eta*np.exp(-X[0])],
                                                 [-gamma*X[1]*np.exp(X[0]),-gamma*(1+np.exp(X[0]))]])

s = lambda t,X,A,gamma: gamma*(A-X[1]-X[1]*X[0])
ds_T = lambda t,X : 0.0
GradS = lambda t,X,gamma: np.array([-gamma*X[1],-gamma*(1+X[0])])

Max_iter = 100
epsilon = 1e-13

In [ ]:
A = 5
a = b = 0.05
gamma, eta = 1e-3, 0.0
args = (A,a,b,gamma,eta)
I0, D0 = 3., 1.2
T0 = 0.820
X0 = np.array([I0,D0])
sol = solve_ivp(fun=f,t_span=[0.0, 10*T0],
                t_eval=np.linspace(0.0,10*T0, 10), 
                y0=X0, method='LSODA', 
                args = args,vectorized=False,
                **{"rtol": 1e-9,"atol":1e-12}
                )
Y = sol.y[:,-1]

In [ ]:
T_ini = 120.
I0, D0 =  2.0, 1.18
X0 = np.array([I0,D0])

In [ ]:
k, Tstar, XX, Norm_B, Norm_DX,M = Newton_orbit(f,s, X0,T_ini,A, a, b, gamma, eta, Gradf,GradS,ds_T, Max_iter, epsilon)

In [ ]:
print(XX[k-1])
Tstar[k-1]

In [ ]:
XX
X0 = XX[k-1]
sol = solve_ivp(fun=f,t_span=[0.0, 14*Tstar[k-1]],
                t_eval=np.linspace(0.0,14*Tstar[k-1], 1000), 
                y0=X0, method='LSODA', 
                args = args,vectorized=False,
                **{"rtol": 1e-9,"atol":1e-12}
                )

## Ploting

In [ ]:
fig, ax = plt.subplots(2,2,sharex='all')
Tab = np.asarray(XX[:k])
ax[0,0].plot(np.arange(k),Tab[:,1],'+-')
ax[0,0].set_ylabel(f"$D_0$")

ax[0,1].plot(np.arange(k),Tab[:,0],'+-')
ax[0,1].set_ylabel(f"$L_0$")
ax[1,0].semilogy(np.arange(k),Norm_DX[:k],'x-')
ax[1,0].set_xlabel("Newton iterations")
ax[1,0].set_ylabel(f"$\parallel \Delta X \parallel$")
ax[1,1].plot(np.arange(k),Tstar[:k],'x-')
ax[1,1].set_xlabel("Newton iterations")
ax[1,1].set_ylabel(f"T")
# ax[1,1].set_ylabel(f"$\parallel (r,s) \parallel_2$")
fig.set_size_inches((8,8))
fig.suptitle(f'Laser model: Newton with a non fixed period \n Parameters: $A = %0.f, a = %.2f,b = %.2f, \eta = %.2f, \gamma = %.3f, T_0 = %.0f, I_0 = %.0f, D_0= %.2f $' % (A, a,b,eta,gamma, T_ini, I0,D0))

fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
plt.savefig('./Results/Laser_model.png')

In [ ]:
X = sol.y
times_intervall = sol.t
plt.plot(times_intervall,X[0,:])
# plt.xlim([0,2*130])
#plt.plot(times_intervall,X[1,:])

plt.xlabel("time")
plt.ylabel("I0")
plt.grid()
plt.show()
# plt.plot(times_intervall,X[:,1])
# plt.ylabel("D0")
# plt.show()
plt.plot(X[0,:], X[1,:])
plt.grid()
X.shape


In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(M)
print("Eigen values of the Monodromy matrix ",np.abs(eigenvalues))
Tab = np.asarray(XX[:k])
plt.plot(np.arange(k), Tab[:,0], label = "I0")
plt.plot(np.arange(k),Tab[:,1], label = "D0")
plt.xlabel("Newton iteration")
plt.ylabel("I0,D0")
plt.legend()
# plt.plot(np.arange(k), Tstar, label = "T")
XX[k-1]